In [1]:
from keras.callbacks import Callback
from keras.objectives import categorical_crossentropy
from keras import backend as K

/home/joshi/xavier/env/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
class GradientActivationStore(Callback):
    
    def __init__(self, filename):
        super(GradientActivationStore, self).__init__()
        self.gradients = []
        self.activations = []
        self.filename = filename
        self.y_true = K.placeholder(shape=[None, 9])
        
    def get_gradients(self, model):
        model_weights = model.trainable_weights
        weights = [weight for weight in model_weights if 'kernel' in weight.name]
        loss = K.mean(categorical_crossentropy(self.y_true, model.output))
        func = K.function([model.input, self.y_true], K.gradients(loss, weights))
        return func
    
    def get_activations(self, model):
        func = K.function([model.input], [layer.output for layer in model.layers[1:]]) # evaluation function
        return func
    
    def on_train_begin(self, logs=None):
        #Gradients
        get_grad = self.get_gradients(self.model)
        inputs = [self.validation_data[0], self.validation_data[1]]
        grads = get_grad(inputs)
        self.gradients.append(grads)
        
        #Activations
        get_act = self.get_activations(self.model)
        acts = get_act([self.validation_data[0]])
        self.activations.append(acts)
        
    def on_epoch_end(self, epoch, logs=None):
        #Gradients
        get_grad = self.get_gradients(self.model)
        inputs = [self.validation_data[0], self.validation_data[1]]
        grads = get_grad(inputs)
        self.gradients.append(grads)
        
        #Activations
        get_act = self.get_activations(self.model)
        acts = get_act([self.validation_data[0]])
        self.activations.append(acts)
        
    def on_train_end(self, logs=None):
        import deepdish as dd
        dd.io.save(self.filename+'-gradients.h5', self.gradients)
        dd.io.save(self.filename+'-activations.h5', self.activations)
        print ('COMPLETE')

In [3]:
VGG16 = keras.applications.vgg16.VGG16(include_top=True, weights=None, input_tensor=None, input_shape=None, pooling=None, classes=1000)


In [9]:
VGG16.layers[1].weights[1].name

u'block1_conv1/bias:0'

In [ ]:
class GradientActivationStore(Callback):
    
    def __init__(self, filename):
        super(GradientActivationStore, self).__init__()
        self.gradients = []
        self.activations = []
        self.filename = filename
        self.y_true = K.placeholder(shape=[None, 9])
        
    def get_gradients(self, model):
        
        model_weights = model.trainable_weights
        weights = [weight for weight in model_weights if 'kernel' in weight.name]
        outputTensor = model.output
        func = K.function([model.input, self.y_true], K.gradients(outputTensor, weights))
        return func
    
    def get_activations(self, model):
        func = K.function([model.input], [layer.output for layer in model.layers[1:]]) # evaluation function
        return func
    
    def on_train_begin(self, logs=None):
        #Gradients
        get_grad = self.get_gradients(self.model)
        inputs = [self.validation_data[0], self.validation_data[1]]
        grads = get_grad(inputs)
        self.gradients.append(grads)
        
        #Activations
        get_act = self.get_activations(self.model)
        acts = get_act([self.validation_data[0]])
        self.activations.append(acts)
        
    def on_epoch_end(self, epoch, logs=None):
        #Gradients
        get_grad = self.get_gradients(self.model)
        inputs = [self.validation_data[0], self.validation_data[1]]
        grads = get_grad(inputs)
        self.gradients.append(grads)
        
        #Activations
        get_act = self.get_activations(self.model)
        acts = get_act([self.validation_data[0]])
        self.activations.append(acts)
        
    def on_train_end(self, logs=None):
        import deepdish as dd
        dd.io.save(self.filename+'-gradients.h5', self.gradients)
        dd.io.save(self.filename+'-activations.h5', self.activations)
        print ('COMPLETE')